In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import joblib

In [ ]:
# Variables
MODEL = 'dense_mlp.keras'
SCALER = 'scaler.save'
LABEL_ENCODER = 'label_encoder.joblib'
CSV_FILE = 'dataset/mlfingerprint_data_unlabeld.csv'

In [ ]:
# Load model, scaler, and label encoder
model = load_model(MODEL)
scaler = joblib.load(SCALER)
le = joblib.load(LABEL_ENCODER)

In [ ]:
# Load unlabeled CSV file
df = pd.read_csv(CSV_FILE)
features = ['packetId', 'length', 'direction', 'iat', 'entropy']
X = df[features].copy()
X['direction'] = X['direction'].map({'INCOMING': 0, 'OUTGOING': 1})
X_scaled = scaler.transform(X.values)

In [ ]:
# Calculate probabilities
y_proba = model.predict(X_scaled)  # shape = (n_samples, n_classes)
mean_probas = y_proba.mean(axis=0)  # average probability per class

labels = list(le.classes_)
sizes = mean_probas

In [ ]:
# Create pie chart
fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(aspect="equal"))
wedges, texts, autotexts = ax.pie(
    sizes,
    labels=labels,
    autopct='%1.1f%%',
    startangle=90,
    counterclock=False
)
ax.set_title("Average Model Probability per Label")
plt.show()